# Notebook to select host galaxies and submit candidates to TNS

In [1]:
import sys, os
import pandas as pd
import numpy as np
import ipyaladin as ipyal
from ipywidgets import Layout, Box, widgets

In [2]:
from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.table import Table, Column

In [3]:
sys.path.append("./lib")
from alerce_tns import *

In [4]:
# continuously load modules (for easier debugging)
%load_ext autoreload
%autoreload 2

# Start API

In [5]:
alerce = alerce_tns()

# Load recent candidates

# CHANGE DATE

In [6]:
refstring = "20191203"
candidates_dir = "candidates"
candidates = open("%s/%s.txt" % (candidates_dir, refstring)).read().splitlines()
print(candidates)

['ZTF19acxguhd', 'ZTF19acxguqy', 'ZTF19acxgwvo', 'ZTF19acxhkvc']


# Select host galaxies using aladin widget

In [7]:
alerce.select_hosts(candidates, catsHTM=False, vizier=False)

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Cannot connect with Simbad...


In [8]:
alerce.candidate_host_names

{'ZTF19acxguhd': 'WHL J223651.7+182247',
 'ZTF19acxguqy': 'WISEA J220158.38+382436.8',
 'ZTF19acxgwvo': 'UM 158',
 'ZTF19acxhkvc': 'SDSS J032920.25+375029.8'}

# Manual modifications

In [9]:
#oid = "ZTF19acmbjll"
#alerce.candidate_host_names[oid] = "WISEA J052037.89-090209.7"
#alerce.candidate_host_redshifts[oid] = "nan"

# Clean nans

In [10]:
for k in list(alerce.candidate_host_redshifts.keys()):
    if alerce.candidate_host_redshifts[k] == "nan":
        alerce.candidate_host_redshifts.pop(k)

# Save hosts

In [11]:
df_host_names = pd.DataFrame.from_dict(alerce.candidate_host_names, orient='index', columns=["host_id"])
df_host_names.index.name='oid'
df_host_redshifts = pd.DataFrame.from_dict(alerce.candidate_host_redshifts, orient='index', columns=["z"])
df_host_redshifts.index.name='oid'
df_hosts = df_host_names.join(df_host_redshifts)
df_hosts.to_csv("candidates/%s_hosts.csv" % refstring)
df_hosts.head(100)

,host_id,z
oid,,
ZTF19acxguhd,WHL J223651.7+182247,0.0761
ZTF19acxguqy,WISEA J220158.38+382436.8,NaN
ZTF19acxgwvo,UM 158,0.014568
ZTF19acxhkvc,SDSS J032920.25+375029.8,0.055


# Display host galaxy properties

In [12]:
for oid in candidates:
    if oid in alerce.candidate_host_redshifts.keys():
        print(oid, alerce.candidate_host_names[oid], alerce.candidate_host_redshifts[oid])
    elif oid in alerce.candidate_host_names.keys():
        print(oid, alerce.candidate_host_names[oid])
    else:
        alerce.candidate_host_names[oid] = ""
        alerce.candidate_host_redshifts[oid] = ""
        print(oid)

ZTF19acxguhd WHL J223651.7+182247 0.0761
ZTF19acxguqy WISEA J220158.38+382436.8
ZTF19acxgwvo UM 158 0.014568
ZTF19acxhkvc SDSS J032920.25+375029.8 0.055


# Sandbox or real

In [41]:
sandbox = False # True: test in sandbox, False: send to real website

# Get API key and url

In [42]:
if sandbox:
    print("Getting API sandbox key...")
    api_key = open("API_sandbox.key", "r").read()
else:
    print("Getting API key...")
    api_key = open("API.key", "r").read()
    
if sandbox:
    url_tns_api="https://sandbox-tns.weizmann.ac.il/api"
else:
    url_tns_api="https://wis-tns.weizmann.ac.il/api"
    
if not sandbox:
    print("WARNING: submitting to real (not sandbox) TNS website")

Getting API key...


# Prepare TNS report

# CHECK AUTHOR LIST!!!
(key is used to query only at this step)

In [43]:
reporter = "F. Forster, F.E. Bauer, G. Pignata, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, F.E. Bauer, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"
report = {"at_report": {}}
counter = 0
for oid in candidates:
    #if oid not in test:#["ZTF19acgjpfd", "ZTF18acszatf", "ZTF18acrkmnm"]:
    #    continue
    #if oid != "ZTF19acmxymo": #ZTF19acnqqhl":#in ["ZTF19acmdwjy", "ZTF19acmdwjz"]:
    #    continue
    print("\n", counter, oid)
    sn_report = alerce.do_TNS_report(api_key, oid, reporter, verbose=False, test=True)
    if sn_report:
        report["at_report"]["%i" % counter] = sn_report
        counter += 1


 0 ZTF19acxguhd
http://alerce.online/object/ZTF19acxguhd


Early SN candidate (g-rise > 0.19+-0.02 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acxguhd 
Astronomical transient is known: [{'objname': '2019vrt', 'prefix': 'AT'}] 


 1 ZTF19acxguqy
http://alerce.online/object/ZTF19acxguqy


Early SN candidate (g-rise > 0.12+-0.02 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acxguqy 

 2 ZTF19acxgwvo
http://alerce.online/object/ZTF19acxgwvo


Early SN candidate (g-rise > 0.22+-0.01 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acxgwvo 

 3 ZTF19acxhkvc
http://alerce.online/object/ZTF19acxhkvc


Early SN candidate (g-rise > 1.34+-0.16 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acxhkvc 


# Display report

In [44]:
print(json.dumps(report, indent=4, sort_keys=True))

{
    "at_report": {
        "0": {
            "at_type": "1",
            "dec": {
                "error": "0.09",
                "units": "arcsec",
                "value": "18.3791227"
            },
            "discovery_data_source_id": "48",
            "discovery_datetime": "2019-12-03 03:06:59.999",
            "host_name": "WHL J223651.7+182247",
            "host_redshift": "0.0761",
            "internal_name": "ZTF19acxguhd",
            "non_detection": {
                "comments": "Data provided by ZTF",
                "exptime": "30",
                "filter_value": "111",
                "flux_units": "1",
                "instrument_value": "196",
                "limiting_flux": "17.9396991729736",
                "obsdate": "2019-12-03 02:38:32.001",
                "observer": "Robot"
            },
            "photometry": {
                "photometry_group": {
                    "0": {
                        "comments": "Data provided by ZTF",
          

# Save report

In [45]:
json_report = '%s/%s.json' % (candidates_dir, refstring)
with open(json_report, 'w') as outfile:
    json.dump(report, outfile)

# Send to TNS

**WARNING: key is used to send candidates!**

In [46]:
json_report

'candidates/20191203.json'

In [47]:
r = alerce.send_json_report(api_key, url_tns_api, json_report)

# Show output

In [48]:
r.content

b'{"id_code":200,"id_message":"OK","data":{"report_id":28253,"recieved_data":{"at_report":{"0":{"ra":{"value":339.2114369,"error":0.09,"units":"arcsec"},"dec":{"value":18.3791227,"error":0.09,"units":"arcsec"},"reporting_group_id":74,"discovery_data_source_id":48,"reporter":"F. Forster, F.E. Bauer, G. Pignata, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, F.E. Bauer, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker","discovery_datetime":"2019-12-03 03:06:59.999","at_type":1,"internal_name":"ZTF19acxguhd","remarks":"Early SN candidate (g-rise > 0.19+-0.02 mag\\/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http:\\/\\/alerce.online\\/object\\/ZTF19acxguhd ","non_detection":{"obsdate":"2019-12-03 02:38:32.001","limiting_flux":17.939699172974,"flux_units":1,"filter_value":111,"instrument_value":

# Check the following link and confirm that the latest SNe are from the reported day

https://wis-tns.weizmann.ac.il/search?discovered_period_value=1&discovered_period_units=days&reporting_groupid%5B%5D=74&discovery_data_source_id=48&num_page=50